In [2]:
import torch
import torchvision
print(torch.cuda.is_available())
print(torch.__version__)

print(torchvision.__version__)
print(torch.version.cuda)

a = torch.Tensor(3, 5)
# a = a.cuda()
print(a)

False
1.13.1
0.14.1
None
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [5]:
import torch
import argparse
import os
import cv2
import torch.backends.cudnn as cudnn
# import models
import torchvision.transforms as transforms
# import flow_transforms
from imageio import imread
from imageio import imsave

# from loss import *
import time
import random
import numpy as np
from glob import glob

import matplotlib.pyplot as plt

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='PyTorch SPixelNet inference on a folder of imgs',
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument('--data_dir', metavar='DIR', default='./demo/inputs', help='path to images folder')
parser.add_argument('--data_suffix',  default='jpg', help='suffix of the testing image')

parser.add_argument('--pretrained', metavar='PTH', help='path to pre-trained model',
                                    default= './pretrain_ckpt/SpixelNet_bsd_ckpt.tar')

parser.add_argument('--output', metavar='DIR', default= './demo' , help='path to output folder')

parser.add_argument('--downsize', default=4, type=float,help='superpixel grid cell, must be same as training setting')

parser.add_argument('--num_threads', default=1, type=int,  help='num_threads')
parser.add_argument('--batch-size', default=1, type=int, metavar='N', help='mini-batch size')
args = parser.parse_args(args=['--data_dir','./demo/inputs','--data_suffix','jpg','--pretrained', 
           './pretrain_ckpt/SpixelNet_bsd_ckpt.tar','--output', './demo','--downsize','4',
           '--num_threads', '1', '--batch-size', '1'])
print(args)
random.seed(100)

In [ ]:
class ArrayToTensor(object):
    """Converts a numpy.ndarray (H x W x C) to a torch.FloatTensor of shape (C x H x W)."""

    def __call__(self, array):
        assert(isinstance(array, np.ndarray))

        array = np.transpose(array, (2, 0, 1))
        # handle numpy array
        tensor = torch.from_numpy(array)
        # put it from HWC to CHW format

        return tensor.float()

In [ ]:
# normalization
input_transform = transforms.Compose([
        ArrayToTensor(),
        transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
        transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
    ])

In [6]:
load_path = 'img/Lena.jpg'
img_ = imread(load_path)[:, :, :3]
# print(img_)

/var/folders/jq/6hj2w9vj1ls5bq_d12lzcyjw0000gn/T/ipykernel_42487/999723470.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_ = imread(load_path)[:, :, :3]


In [ ]:
def shift9pos(input, h_shift_unit=1,  w_shift_unit=1):
    # input should be padding as (c, 1+ height+1, 1+width+1)
    input_pd = np.pad(input, ((h_shift_unit, h_shift_unit), (w_shift_unit, w_shift_unit)), mode='edge')
    input_pd = np.expand_dims(input_pd, axis=0)

    # assign to ...
    top     = input_pd[:, :-2 * h_shift_unit,          w_shift_unit:-w_shift_unit]
    bottom  = input_pd[:, 2 * h_shift_unit:,           w_shift_unit:-w_shift_unit]
    left    = input_pd[:, h_shift_unit:-h_shift_unit,  :-2 * w_shift_unit]
    right   = input_pd[:, h_shift_unit:-h_shift_unit,  2 * w_shift_unit:]

    center = input_pd[:,h_shift_unit:-h_shift_unit,w_shift_unit:-w_shift_unit]

    bottom_right    = input_pd[:, 2 * h_shift_unit:,   2 * w_shift_unit:]
    bottom_left     = input_pd[:, 2 * h_shift_unit:,   :-2 * w_shift_unit]
    top_right       = input_pd[:, :-2 * h_shift_unit,  2 * w_shift_unit:]
    top_left        = input_pd[:, :-2 * h_shift_unit,  :-2 * w_shift_unit]

    shift_tensor = np.concatenate([     top_left,    top,      top_right,
                                        left,        center,      right,
                                        bottom_left, bottom,    bottom_right], axis=0)
    return shift_tensor


In [ ]:
H, W, _ = img_.shape
# ceil向上取整
# H_, W_  = int(np.ceil(H/16.)*16), int(np.ceil(W/16.)*16)
H_, W_  = int(np.ceil(H/4.)*4), int(np.ceil(W/4.)*4)
n_spixl_h = int(np.floor(H_ / args.downsize))
n_spixl_w = int(np.floor(W_ / args.downsize))
print(H_, W_) # 32*32
print('downsize=', args.downsize) # downsize=4.0
print('spixl-h', n_spixl_h) # superpixel-h=8
print('spixl-w', n_spixl_w) # superpixel-w=8

spix_values = np.int32(np.arange(0, n_spixl_w * n_spixl_h).reshape((n_spixl_h, n_spixl_w)))
spix_idx_tensor_ = shift9pos(spix_values)
print('spixel_values', spix_values.shape, '\n', spix_values) # shape=8*8
print('after shifting position')
print('spixel_idx_tensor',spix_idx_tensor_.shape, '\n', spix_idx_tensor_) # tensor shape=(9,8,8)


In [ ]:
# 在宽度和高度维上进行复制，downsize是下采样倍数
spix_idx_tensor = np.repeat(
      np.repeat(spix_idx_tensor_, args.downsize, axis=1), args.downsize, axis=2)
print('spix_idx_tensor_', spix_idx_tensor_.shape, '\n', spix_idx_tensor_)
# 9，H/downsize，W/downsize
# 9， 2H/downsize，2W/downsize


In [ ]:
# 超像素个数
n_spixel =  int(n_spixl_h * n_spixl_w)
# 扩张成4dim tensor，分别是pic数量1，超像素索引，高度，宽度
spixeIds = torch.from_numpy(np.tile(spix_idx_tensor, (1, 1, 1, 1))).type(torch.float).cuda()
# from_numpy 将数组变成tensor并且改变tensor的值，array值也会变
print('n_spixel', n_spixel) 
# n_pixel = 64
print('spixeIds', spixeIds.shape, '\n', spixeIds)
#  torchsize=([1, 9, 32, 32])


In [ ]:
# updating superpixel index

def update_spixl_map (spixl_map_idx_in, assig_map_in):
    assig_map = assig_map_in.clone()

    b,_,h,w = assig_map.shape
    _, _, id_h, id_w = spixl_map_idx_in.shape

    print('output',h, w )
    print('spixl', id_h, id_w)
    # 当前像素位置hw与给定的超像素索引，相同将super pixel index赋值给map idx
    if (id_h == h) and (id_w == w):
        spixl_map_idx = spixl_map_idx_in
    else:
        # 不同需要上采样or下采样调整为in
        spixl_map_idx = F.interpolate(spixl_map_idx_in, size=(h,w), mode='nearest')
    # 计算分配概率assign map中超像素最大概率作为该超像素的超像素标签
    assig_max,_ = torch.max(assig_map, dim=1, keepdim= True)
    # 属于标签的设1，否则0
    assignment_ = torch.where(assig_map == assig_max, torch.ones(assig_map.shape).cuda(),torch.zeros(assig_map.shape).cuda())
    # 更新超像素标签
    new_spixl_map_ = spixl_map_idx * assignment_ # winner take all
    # new_spixl_map = torch.sum(new_spixl_map_,dim=1,keepdim=True).type(torch.int)

    return new_spixl_map_, assignment_

In [ ]:

# function： 得到超像素分割图，以及进行enforce connect约束后超像素索引

def get_spixel_image(given_img, spix_index, n_spixels = 600, b_enforce_connect = False):
    # enforce_connectivity如果设置成truth能够减少生成的数量，结合更紧密
    if not isinstance(given_img, np.ndarray):
        given_img_np_ = given_img.detach().cpu().numpy().transpose(1,2,0)
    else: # for cvt lab to rgb case
        given_img_np_ = given_img

    if not isinstance(spix_index, np.ndarray):
        spix_index_np = spix_index.detach().cpu().numpy().transpose(0,1)
    else:
        spix_index_np = spix_index

    h, w = spix_index_np.shape
    given_img_np = cv2.resize(given_img_np_, dsize=(w, h), interpolation=cv2.INTER_CUBIC)

    if b_enforce_connect:
        spix_index_np = spix_index_np.astype(np.int64)
        segment_size = (given_img_np_.shape[0] * given_img_np_.shape[1]) / (int(n_spixels) * 1.0)
        min_size = int(0.06 * segment_size)
        max_size =  int(3 * segment_size)
        # index包含所有像素所属的超像素的索引，min和max是限制超像素最大最小尺寸参数
        spix_index_np = enforce_connectivity(spix_index_np[None, :, :], min_size, max_size)[0]
    cur_max = np.max(given_img_np)
    # 先/curmax像素归一化，超像素index转化为int类型，边缘颜色定为青色011
    spixel_bd_image = mark_boundaries(given_img_np/cur_max, spix_index_np.astype(int), color = (0,1,1)) #cyna
    return (cur_max*spixel_bd_image).astype(np.float32).transpose(2,0,1), spix_index_np #
    # 返回超像素边界图，以及index用于后续任务中使用


In [ ]:
from connectivity import enforce_connectivity
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

img = cv2.resize(img_, (W_, H_), interpolation=cv2.INTER_CUBIC)
# normalization
img1 = input_transform(img)
ori_img = input_transform(img_)
torch.set_printoptions(profile="full")
print('input_transform img', img1.shape, '\n') # 3*32*32

# tensor[3，16，16]

# output = model(img1.cuda().unsqueeze(0))
# unsqueeze(0)将数据扩充维度，理解为给数据的索引，方便网络对数据进行批处理batch
# output tensor [1,9,16,16]

output = torch.load('2_32_4.pt')
# assign the spixel map
print('spixeIds', spixeIds.shape) # 1*9*32*32
print('output', output.shape) # 1*9*32*32
curr_spixl_map, assignment_ = update_spixl_map(spixeIds, output)
# ori_sz_spixel_map = F.interpolate(curr_spixl_map.type(torch.float), size=( H_,W_), mode='nearest').type(torch.int)
# 如果size合规的话curr_spixel_map就等于ori_sz_spixel_map
mean_values = torch.tensor([0.411, 0.432, 0.45], dtype=img1.cuda().unsqueeze(0).dtype).view(3, 1, 1)
print('mean_values', mean_values.shape, '\n', mean_values) # 3*1*1

# get_spixel_image参数分别是
# spixel_viz, spixel_label_map = get_spixel_image((ori_img + mean_values).clamp(0, 1), curr_spixl_map.squeeze(), n_spixels= n_spixel,  b_enforce_connect=True)

# clamp将输入的normalization的tensor压缩到0-1之间，输出一个新的tensor
# squeeze 将维度为1的删除


print('assignment', assignment_.shape)
# shape: 1,9,32,32
# 将assignment可视化steps：1.压缩维度，2.选取需要维度，3.转化为数组，4.可视化
assignment = torch.squeeze(assignment_)
# squeeze去掉一维
# shape: 9,32,32
# assignment = torch.sum(assignment_,dim=0,keepdim=True).type(torch.int)
'''
indices = torch.tensor([0])
torch.index_select(assignment, 0, indices)
'''
for i in range(0, 1):
    assignment = assignment[i,:,:]
    assignment = assignment.cpu().numpy()
    # plt.matshow(assignment, cmap=plt.cm.Blues)
    ax = plt.matshow(assignment)
    plt.colorbar(ax.colorbar)
    plt.title("assignment map")
    plt.show()

# print('curr_spixl_map', curr_spixl_map.shape, '\n', curr_spixl_map)

curr_spixl_map = torch.squeeze(curr_spixl_map)
curr_spixl_map = curr_spixl_map[0,:,:]
print('curr_spixl_map shape', curr_spixl_map.shape)
curr_spixl_map = curr_spixl_map.cpu().numpy()
# plt.matshow(assignment, cmap=plt.cm.Blues)
ax = plt.matshow(curr_spixl_map)
plt.colorbar(ax.colorbar)
plt.title("cur_spixl map")
plt.show()


print('assignment', assignment_.shape, '\n', assignment_)

print('get final spixel map')
# print('spixel_viz', spixel_viz.shape)